In [ ]:
# Go to directory: soccer-stars-django-webapp/server/backend
# Run in terminal: daphne -p 8000 backend.asgi:application

In [12]:
import nest_asyncio
nest_asyncio.apply()
import numpy as np
from typing import List
import asyncio
import json
from websocket import create_connection
from websocket import WebSocket

In [13]:
import os
os.chdir("/home/david/Documents/data_science/projects/soccer-stars-django-webapp/server/backend")

In [14]:
from match.match_logic import game_entities, formations, motion, utils

In [15]:
# Create a connection to the WebSocket
ws = create_connection("ws://localhost:8000/ws/match/")

In [16]:
def test_get_formation(ws: WebSocket):
    

    # Send the initial formation payload
    formation_payload = {
        "action": "create_formation",
        "left_formation": "formation1",
        "right_formation": "formation2"
    }
    ws.send(json.dumps(formation_payload))

    # Receive the response for initial formation
    result = ws.recv()

    # Parse the response to get the initial positions
    response_data = json.loads(result)

    return response_data

def test_arrow(ws: WebSocket, cap_idx, arrow_power, angle, initial_positions: List):
    """
        # Send the arrow submission payload
    arrow_payload = {
        "action": "submit_arrow",
        "cap_idx": 0,
        "arrow_power": 190,
        "angle": 15,
        "positions": initial_positions
    }
    ws.send(json.dumps(arrow_payload))

    # Receive the response for the arrow submission
    result = ws.recv()
    print("Received arrow submission: '%s'" % result)

    # Close the WebSocket connection
    ws.close()
    """
    
    # Create a connection to the WebSocket
    ws = create_connection("ws://localhost:8000/ws/match/")

    # Send the arrow submission payload
    arrow_payload = {
        "action": "submit_arrow",
        "cap_idx": cap_idx,
        "arrow_power": arrow_power,
        "angle": angle,
        "positions": initial_positions
    }
    ws.send(json.dumps(arrow_payload))

    # Receive the response for the arrow submission
    result = ws.recv()

    return json.loads(result)

In [17]:
# ---------------------------------- #
#   Start Game
# ---------------------------------- #
json_initial_positions = test_get_formation(ws=ws)
initial_positions = json_initial_positions.get("initial_positions")

In [18]:
%%time
json_xhist = test_arrow(
    ws=ws, 
    cap_idx=4,
    arrow_power=190,
    angle=15,
    initial_positions=initial_positions
)

CPU times: user 53.2 ms, sys: 1.32 ms, total: 54.5 ms
Wall time: 117 ms


In [19]:
Xhist = json_xhist.get("positions")

In [20]:
json_xhist = test_arrow(
    ws=ws, 
    cap_idx=6,
    arrow_power=190,
    angle=210,
    initial_positions=Xhist[-1]
)
Xhist2 = json_xhist.get("positions")

# Visualize Game

In [21]:
w = 1920
h = 1080
margin = 300
boundaries = (w, h)
ratio_radii_cap_h = 1./17
radii = int(h * ratio_radii_cap_h)
goal_size = int(radii * 6)
goal_depth = int(radii * 2)
ur = utils.UtilsRender(window_size=boundaries, goal_depth=goal_depth, goal_size=goal_size)

In [22]:
n = len(initial_positions)
radii = 63
radii_ball = 31
R = np.array([radii for _ in range(n)])
R[-1] = radii_ball
ur.render_field_motion(positions=Xhist2, radius=R, add_delay=3)